In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

In [2]:
#Import required sklearn functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.inspection import permutation_importance
from collections import defaultdict

In [3]:
#Import sklearn classifiers
from sklearn.ensemble import RandomForestClassifier

In [4]:
#Import library to oversample 
from imblearn.over_sampling import RandomOverSampler

In [5]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [6]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [7]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [8]:
#Functions used in the study

#Remove those numbers from analysis data
def filter_rows_by_values1(df, col, values):
    return df[~df[col].isin(values)]

#Remove those numbers from analysis data
def filter_rows_by_values2(df, col, values):
    return df[df[col].isin(values)]

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

def remove_95correlated(correlated):
    #Remove any features that are greater than 95% correlated
    corr_matrix = correlated.corr()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

    correlated = correlated.drop(to_drop, axis = 1)
    corr_matrix = correlated.corr()
    return correlated

def remove_nonimportant(X_values, y_values):
    # Specifys Random Forest and the Number of Trees, SelectFromModel will
    # select features which are most important
    feature_names = [f"feature {i}" for i in range(X_values.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(X_values, y_values)

    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time

    threshold = np.sort(importances)[-100]
    
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(X_values, y_values)

    # Select the final features set 
    sel.get_support()
    selected_feat= X_values.columns[(sel.get_support())]

    # Prints the names of the final selected features
    print(selected_feat)
    X_values = X_values[selected_feat]
    
    return X_values

def dendrogram(X_values, y):
    corr = spearmanr(X_values).correlation
    # Ensure the correlation matrix is symmetric
    corr = (corr + corr.T) / 2
    np.fill_diagonal(corr, 1)
    distance_matrix = 1 - np.abs(corr)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
  
    trained_cluster_ids = hierarchy.fcluster(dist_linkage, y, criterion="distance")
    trained_cluster_id_to_feature_ids = defaultdict(list) 
    for idx, trained_cluster_id in enumerate(trained_cluster_ids):
        trained_cluster_id_to_feature_ids[trained_cluster_id].append(idx)
    
    trained_selected_features = [v[0] for v in trained_cluster_id_to_feature_ids.values()]
    final_selected_features = X_values.columns[trained_selected_features]
    X_train = X_values[final_selected_features]
    return X_train

def classificationMetrics(results, y_test, pred):
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred, average=None, zero_division=0)
    recall = recall_score(y_test, pred, average=None)
    F1 = f1_score(y_test, pred, average=None)           
    #Calculate confusion matrix
    cf_matrix = confusion_matrix(y_test, pred)
    cf_matrix = np.reshape(cf_matrix,(1,4))
    comb = np.concatenate((x, y, cf_matrix, acc, prec, recall, F1), axis=None)
    comb = [comb]
    results = results.append(pd.DataFrame(comb, columns=results.columns), ignore_index=True)
    return results


In [9]:
# Read Training/Test data input File
data = pd.read_csv('BorylationTrainingTest 1-10-25.csv')

#group the compounds by numbers
data['grouped'] = data.groupby('Substrate', sort=False).ngroup()
grouped_sub = data[['grouped','Substrate','Product_Ratio']]
#Convert substrates to Mordred features
mordred_data = get_Mordred(grouped_sub)

mordred_data.head()

100%|██████████| 971/971 [01:24<00:00, 11.43it/s]


,grouped,Substrate,Product_Ratio,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,VE3_A,VR1_A,VR2_A,VR3_A,nAromAtom,nAromBond,nAtom,nHeavyAtom,nSpiro,nBridgehead,...,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW02,SRW03,SRW04,SRW05,SRW06,SRW07,SRW08,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,0.0,1,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,2.876615,2.673468,0.334183,0.760233,21.482988,2.685374,2.844118,0,0,26,8,0,0,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
1,0,0.0,0,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,2.876615,2.673468,0.334183,0.760233,21.482988,2.685374,2.844118,0,0,26,8,0,0,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
2,0,0.0,0,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,2.876615,2.673468,0.334183,0.760233,21.482988,2.685374,2.844118,0,0,26,8,0,0,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
3,0,0.0,0,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,2.876615,2.673468,0.334183,0.760233,21.482988,2.685374,2.844118,0,0,26,8,0,0,...,5.129899,5.758902,6.385194,7.016610,7.645398,8.277158,66.911677,2.708050,0.0,3.663562,0.0,4.762174,0.0,5.926926,0.0,7.126891,32.187603,114.140851,4.390033,84,5,26.0,24.0,3.500000,2.25
4,1,0.0,1,5.656854,5.427660,0,1,10.424292,2.135779,4.271558,10.424292,1.303037,2.969338,2.718002,0.339750,0.776754,23.560979,2.945122,2.936448,0,0,23,8,0,0,...,5.796058,6.555357,7.311218,8.071843,8.828348,9.589256,75.041313,2.833213,0.0,3.970292,0.0,5.342334,0.0,6.803505,0.0,8.298291,35.247635,113.120449,4.918280,64,7,34.0,36.0,2.611111,2.00


In [10]:
Results_df = pd.DataFrame(columns =  ['x', 'y',  "True Neg","False Pos","False Neg","True Pos",'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

maxacc_comb = pd.DataFrame()
val_tot = pd.DataFrame()
prod = pd.DataFrame()
test_index_total = pd.DataFrame()

model_columns = pd.DataFrame()
for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 188,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(mordred_data, "grouped", set_numbers)
    test_data = filter_rows_by_values2(mordred_data, "grouped", set_numbers)
  
    #Remove features that dont change
    training_data = remove_zero_varience(training_data)
    
    #Remove features that are more than 95% correlated
    training_data = remove_95correlated(training_data)
    
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio', axis = 1)
    training_X = training_X.drop('grouped', axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio', axis = 1)
    test_X = test_X.drop('grouped', axis = 1)
    test_y = test_data['Product_Ratio']
 
    #Remove features that are considered less important
    feature_names = [f"feature {i}" for i in range(training_X.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(training_X, training_y)
    
    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    
    threshold = np.sort(importances)[-100] 
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(training_X, training_y)
     
    # Select the reduced features set 
    sel.get_support()
    selected_feat= training_X.columns[(sel.get_support())]
    
    reduced1_X = training_X[selected_feat]
    test_X = test_X[selected_feat]
    
    #Apply over-sampling to dataset
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(reduced1_X, training_y) 
    
    for y in [ 
              0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35,
              0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 
              0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
              0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50]:
             
    
        #Make final training and test set and save them as df's  
        X_train = dendrogram(X_resampled, y)
        test_X = test_X[X_train.columns]
        training_columns_list = X_train.columns.tolist()
        training_columns_list = (x, y, training_columns_list)
        training_columns_list = (pd.DataFrame(training_columns_list).T)

        #Random Forest Classifier
        rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
        rfc.fit(X_train, y_resampled)
        pred = rfc.predict(test_X)
        Results_df = classificationMetrics(Results_df, test_y, pred)

        #Evaluate model by going line by line
        ynew = rfc.predict(test_X)
        prediction_df = pd.DataFrame(ynew,  columns = [(x,y)])

        val_pred_T = prediction_df.T
        val_tot = val_tot.append(val_pred_T)

        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = Results_df.groupby('y')['acc'].mean()
        acc_std = Results_df.groupby('y')['acc'].std()
        precision_0_mean = Results_df.groupby('y')['precision 0'].mean()
        precision_0_std = Results_df.groupby('y')['precision 0'].std()
        precision_1_mean = Results_df.groupby('y')['precision 1'].mean()
        precision_1_std = Results_df.groupby('y')['precision 1'].std()
        recall_0_mean = Results_df.groupby('y')['recall 0'].mean()
        recall_0_std = Results_df.groupby('y')['recall 0'].std()
        recall_1_mean = Results_df.groupby('y')['recall 1'].mean()
        recall_1_std = Results_df.groupby('y')['recall 1'].std()
        F1_0_mean = Results_df.groupby('y')['F1 0'].mean()
        F1_0_std = Results_df.groupby('y')['F1 0'].std()
        F1_1_mean = Results_df.groupby('y')['F1 1'].mean()
        F1_1_std = Results_df.groupby('y')['F1 1'].std()
        true_neg_mean = Results_df.groupby('y')['True Neg'].mean()
        true_neg_std = Results_df.groupby('y')['True Neg'].std()
        false_pos_mean = Results_df.groupby('y')['False Pos'].mean()
        false_pos_std = Results_df.groupby('y')['False Pos'].std()        
        false_neg_mean = Results_df.groupby('y')['False Neg'].mean()
        false_neg_std = Results_df.groupby('y')['False Neg'].std()      
        true_pos_mean = Results_df.groupby('y')['True Pos'].mean() 
        true_pos_std = Results_df.groupby('y')['True Pos'].std()   
        

        average_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std,
                                   true_neg_mean, true_neg_std,
                                   false_pos_mean, false_pos_std,
                                   false_neg_mean, false_neg_std,
                                   true_pos_mean, true_pos_std], axis=1)
    
        average_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std', 'true_neg_mean', 'true_neg_std',
                                 'false_pos_mean', 'false_pos_std','false_neg_mean', 'false_neg_std',
                                 'true_pos_mean', 'true_pos_std']                                 

        maxacc = average_df[average_df.acc_mean == average_df.acc_mean.max()]
        maxacc_copy  = maxacc.copy()
        maxacc_copy['x_col'] = x
        
        model_columns = model_columns.append(training_columns_list)
        #print(x,y)
    test_index = pd.DataFrame(test_data.index.values)
    test_index_total = pd.concat([test_index_total, test_index],axis = 1)
    test_y = test_y.rename(x)
    prod = prod.append(test_y)
    maxacc_comb = maxacc_comb.append(maxacc_copy)  

total_results = val_tot.T
 
maxacc_comb.to_csv("Mordredonly.csv")
model_columns = model_columns.rename(columns = {0:'x', 1:'y', 2: 'features'})
model_columns = model_columns.drop_duplicates(subset = ['x','y'])
model_columns.to_csv("Mordredonly.csv", mode="a")

In [11]:
maxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,true_neg_mean,true_neg_std,false_pos_mean,false_pos_std,false_neg_mean,false_neg_std,true_pos_mean,true_pos_std,x_col
y,,,,,,,,,,,,,,,,,,,,,,,
0.35,0.660309,0.012961,0.848002,0.001336,0.271309,0.005859,0.707006,0.020585,0.462162,0.019942,0.770955,0.011870,0.341621,0.002755,111.000000,3.231787,46.000000,3.231787,19.900000,0.737865,17.100000,0.737865,1
0.35,0.607900,0.055193,0.844433,0.003878,0.270130,0.004859,0.621518,0.089974,0.544595,0.086484,0.712719,0.061198,0.358973,0.017924,91.950000,19.765667,54.550000,9.253591,16.850000,3.199918,20.150000,3.199918,2
0.35,0.617119,0.047600,0.842025,0.004893,0.273788,0.007399,0.638355,0.078539,0.527928,0.076025,0.723548,0.052842,0.358634,0.014702,93.333333,16.297521,52.000000,8.674179,17.466667,2.812942,19.533333,2.812942,3
0.50,0.634066,0.055213,0.843617,0.006451,0.263135,0.019492,0.667296,0.091758,0.483784,0.105619,0.741708,0.061192,0.338224,0.036548,103.600000,23.542433,49.900000,9.529442,19.100000,3.907914,17.900000,3.907914,4
0.35,0.647677,0.053457,0.846374,0.006776,0.263244,0.017070,0.687188,0.086434,0.466486,0.096881,0.755679,0.057331,0.334178,0.033370,109.680000,23.876758,48.120000,8.572952,19.740000,3.584604,17.260000,3.584604,5
0.35,0.641583,0.050710,0.844458,0.007543,0.263982,0.015670,0.678717,0.081216,0.474324,0.090179,0.750091,0.053841,0.337120,0.031129,106.566667,22.875764,48.766667,7.981688,19.450000,3.336623,17.550000,3.336623,6
0.35,0.627547,0.058688,0.843933,0.007237,0.262177,0.015325,0.656683,0.093275,0.494595,0.097540,0.735163,0.062435,0.339942,0.029681,102.057143,23.945521,51.514286,10.127796,18.700000,3.608966,18.300000,3.608966,7
0.35,0.627339,0.055028,0.840160,0.012114,0.263953,0.015201,0.659184,0.087736,0.488176,0.092965,0.735611,0.058523,0.339973,0.027753,100.550000,22.756777,50.450000,9.928733,18.937500,3.439693,18.062500,3.439693,8
0.35,0.625046,0.052281,0.842066,0.012653,0.266635,0.016230,0.653156,0.084483,0.502102,0.096216,0.732686,0.055797,0.345757,0.030898,98.922222,21.939256,51.077778,9.534359,18.422222,3.560008,18.577778,3.560008,9


In [12]:
final_selected_features = ['VR1_A', 'ATS0p', 'AATS5p', 'AATS3i', 'AATSC6c', 'AATSC2d', 'AATSC4d', 'AATSC3s', 'AATSC5Z', 'AATSC3v', 
                           'AATSC5v', 'AATSC4p', 'MATS5s', 'MATS6s', 'MATS3Z', 'MATS6Z', 'MATS6p', 'MATS1i', 'GATS2c', 'GATS4c', 'GATS7c', 'GATS5i', 'BCUTdv-1l', 'RPCG', 'AXp-4dv', 'ETA_dEpsilon_C', 'IC3', 'SIC0', 'FilterItLogS', 'MDEC-22']



#Loads validation dataset for borlation using the final reduced features 
unknownSubstrates=pd.read_csv('validation1-9-25.csv')

# Convert validation substrates Inchi's to Mordred and combine into Dataframe with atomic charges and JChem paramters
New_Substrate = unknownSubstrates['Substrate']
New_Substrate_mol_list = []
for inChi_New_Substrate in New_Substrate:
  New_Substrate_mol = Chem.MolFromInchi(inChi_New_Substrate)
  New_Substrate_mol_list.append(New_Substrate_mol)

New_Substrate_data = []
New_Substrate_data = calc.pandas(New_Substrate_mol_list)
New_Substrate_data = New_Substrate_data.apply(pd.to_numeric, errors='coerce')
New_Substrate_data.fillna(0, inplace=True)                                                                  
XnewSec = pd.concat((unknownSubstrates, New_Substrate_data), axis=1)
Xnew = XnewSec[final_selected_features]

val_tot = pd.DataFrame()

for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 188,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(mordred_data, "grouped", set_numbers)
    test_data = filter_rows_by_values2(mordred_data, "grouped", set_numbers)
   
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']
   
    #Apply over-sampling to training set
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(training_X, training_y)    
    X_train = X_resampled[final_selected_features]

    #Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=800,max_depth=9)
    rfc.fit(X_train, y_resampled)
    
    #Evaluate the model on validation set
    ynew = rfc.predict(Xnew)
    validation_prediction_df = pd.DataFrame(ynew, columns = [(x)])
    validation_prediction_df.merge(validation_prediction_df, on=x)
    val_pred_T = validation_prediction_df.T
    val_tot = val_tot.append(val_pred_T)        

#Print the validation evaluations for model
unknownSubstrates_prod = unknownSubstrates['Product_Ratio']
total_val_results_transposed = val_tot.T
Val_results = pd.concat((unknownSubstrates_prod, total_val_results_transposed), axis=1)

#Write the results onto a CSV file 
#totalResults_df.to_csv("10Runs_FullResults.csv", index=False)
Val_results.to_csv("Mordredonly.csv", mode="a")
Val_results

100%|██████████| 81/81 [00:20<00:00,  3.98it/s]


,Product_Ratio,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
76,1,1,1,1,1,1,1,1,1,1,1
77,0,1,1,1,1,1,1,1,1,1,1
78,0,1,1,1,1,1,1,1,1,1,1
79,0,1,1,1,1,1,1,1,1,1,1
